# [Digital Archive of Queensland Architecture](https://qldarch.net/) Data Scraping

## 0. Setting

In [1]:
import copy
import os
import pprint

from collections import defaultdict

import pandas as pd
from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=2)

In [2]:
import sys

codefolder = "C:/ProjectCollections/Programs/Australia_Cultural_Data_Engine/codes"

data_folder = "D:/Program_Data/Australia_Cultural_Data_Engine_Data/digital_archive_of_queensland_architecture"

sys.path.append(codefolder)

from daqa import DataManipulation as daqa_dbmanip
from daqa import WebScraping as daqa_webscrap

daqa_scrap = daqa_webscrap.WebScraping()

## 1. Set Up MongoDB Database for DAQA Scraping

### 1.1 Create/Connect DAQA Database & Collections

[How do I create a new database in MongoDB using PyMongo?](https://stackoverflow.com/questions/8566618/how-do-i-create-a-new-database-in-mongodb-using-pymongo/42039275#42039275)

### 1.2 Drop DAQA Database & Collections

In [3]:
# daqa_scrap.daqa_db.drop_collection(collection_name)
daqa_scrap.localclient.drop_database('daqa_scraped')

# mycol = mydb["daqa_scraped"]
# collist = mydb.list_collection_names()

## 2. Scrap All Records

https://qldarch.net/ws/search?

[MongoDB SELECT COUNT GROUP BY](https://stackoverflow.com/questions/23116330/mongodb-select-count-group-by)

### 2.1 Scraping and Storing

In [4]:
# erase all records in objects collection
daqa_scrap.all_obj_coll.drop()
daqa_scrap.relationship_coll.drop()

daqa_scrap.store_daqa_objs(query_terms="q=*", page_count=1000)

store_objects: 100%|████████████████████████████████████████████████████████████| 12589/12589 [00:17<00:00, 699.59it/s]


### 2.2 Type Checking

In [5]:
objects_summary = list(
    daqa_scrap.all_obj_coll.aggregate(
        [
            {
                "$group": {
                    "_id": {"category": "$category", "type": "$type"},
                    "count": {"$sum": 1},
                }
            },
            {"$sort": {"_id.category": 1, "count": -1}},
        ]
    )
)
objects_summary = pd.json_normalize(objects_summary).set_index(
    ["_id.category", "_id.type"]
)
objects_summary

count
_id.category _id.type           
archobj      structure      2163
             person         1099
             firm            901
             article         783
             interview        91
             publication      46
             topic            44
             education        39
             award            27
             event            21
             place            19
             organisation     15
             government        6
media        Photograph     4016
             LineDrawing    1152
             Image          1007
             Article         686
             Audio           142
             Transcript      128
             Portrait        113
             Youtube          46
             Video            42
             Spreadsheet       3

## 3. Scrap Key Objects

### Persons (and Their Relationships) Scraping & Storing

In [6]:
daqa_scrap.person_coll.drop()
print(f"All records in {daqa_scrap.person_coll.name} collection are erased!")

daqa_scrap.store_keyobjs(
    obj_name="architects", obj_query=["person"], obj_coll=daqa_scrap.person_coll,
)

All records in person collection are erased!


architects: 100%|█████████████████████████████████████████████████████████████████▉| 1100/1101 [03:17<00:00,  5.58it/s]


### Firms  (and Their Relationships) Scraping & Storing

In [7]:
daqa_scrap.firm_coll.drop()
print(f"All records in {daqa_scrap.firm_coll.name} collection are erased!")

daqa_scrap.store_keyobjs(
    obj_name="firms", obj_query=["firm"], obj_coll=daqa_scrap.firm_coll,
)

All records in firm collection are erased!


firms: 100%|█████████████████████████████████████████████████████████████████████████| 904/904 [02:28<00:00,  6.11it/s]


### Projects  (and Their Relationships) Scraping & Storing

In [8]:
daqa_scrap.stru_coll.drop()
print(f"All records in {daqa_scrap.stru_coll.name} collection are erased!")

daqa_scrap.store_keyobjs(
    obj_name="projects", obj_query=["structure"], obj_coll=daqa_scrap.stru_coll,
)

All records in structure collection are erased!


projects: 100%|████████████████████████████████████████████████████████████████████| 2175/2175 [05:47<00:00,  6.26it/s]


In [9]:
with tqdm(total=10 ** 4, desc="add_location_details") as t:
    for stru in daqa_scrap.stru_coll.find(
        {"latitude": {"$exists": 1}, "longitude": {"$exists": 1}},
        {"latitude": 1, "longitude": 1},
    ):
        location_details = daqa_scrap.get_loc(
            stru.get("latitude"), stru.get("longitude")
        )
        if location_details:
            daqa_scrap.stru_coll.update_one(
                {"_id": stru["_id"]},
                {"$set": {"coord_loc_details": location_details.raw}},
            )
            t.update(1)

add_location_details:  14%|███████▋                                             | 1447/10000 [12:04<1:11:20,  2.00it/s]


### Interviews  (and Their Relationships) Scraping & Storing

In [10]:
daqa_scrap.interview_coll.drop()
print(f"All records in {daqa_scrap.interview_coll.name} collection are erased!")

daqa_scrap.store_keyobjs(
    obj_name="interviews", obj_query=["interview"], obj_coll=daqa_scrap.interview_coll,
)

All records in interview collection are erased!
https://qldarch.net/ws/interviews: 500


interviews: 100%|██████████████████████████████████████████████████████████████████████| 91/91 [00:23<00:00,  3.94it/s]


### Articles (and Their Relationships) Scraping & Storing

In [11]:
daqa_scrap.article_coll.drop()
print(f"All records in {daqa_scrap.article_coll.name} collection are erased!")

daqa_scrap.store_keyobjs(
    obj_name="articles", obj_query=["article"], obj_coll=daqa_scrap.article_coll,
)

All records in article collection are erased!


articles: 100%|██████████████████████████████████████████████████████████████████████| 783/783 [01:42<00:00,  7.67it/s]


## 4. Records Supplement

### Add Other Archival Objects Existing in `all_obj_coll`

In [12]:
for other_obj in daqa_scrap.all_obj_coll.find({"category": "archobj"}).distinct("type"):
    if other_obj not in daqa_scrap.key_archobjs:
        daqa_scrap.daqa_db[other_obj].drop()
        daqa_scrap.store_keyobjs(
            obj_name=f"{other_obj}s",
            obj_query=[other_obj],
            obj_coll=daqa_scrap.daqa_db[other_obj],
        )

https://qldarch.net/ws/awards: 404


awards: 100%|██████████████████████████████████████████████████████████████████████████| 27/27 [00:04<00:00,  5.68it/s]


https://qldarch.net/ws/educations: 404


educations: 100%|██████████████████████████████████████████████████████████████████████| 39/39 [00:07<00:00,  5.39it/s]


https://qldarch.net/ws/events: 404


...

places: 100%|██████████████████████████████████████████████████████████████████████████| 19/19 [00:02<00:00,  6.49it/s]


https://qldarch.net/ws/publications: 404


publications: 100%|████████████████████████████████████████████████████████████████████| 46/46 [00:05<00:00,  7.76it/s]


https://qldarch.net/ws/topics: 404


topics: 100%|██████████████████████████████████████████████████████████████████████████| 44/44 [00:06<00:00,  6.74it/s]


### Add Media Existing in `all_obj_coll`

In [13]:
daqa_scrap.daqa_db["media"].drop()
print(f"All records in media collection are erased!")

media_ids = list(daqa_scrap.all_obj_coll.find({"category": "media"}).distinct("id"))
for media_id in tqdm(media_ids, total=len(media_ids), desc="media", leave=True):
    media_record = daqa_scrap.all_obj_coll.find_one(
        {"category": "media", "id": media_id}, {"_id": 0}
    )
    # media_record["media_type"] = media_record.get("type")
    # media_record.pop("type")
    daqa_scrap.daqa_db["media"].insert_one(media_record)

All records in media collection are erased!


media: 100%|██████████████████████████████████████████████████████████████████████| 6775/6775 [01:00<00:00, 112.65it/s]


### Relationships Supplement

#### Add Firms' Relationships

This section extracts `PrecededBy` and `SucceededBy` relationships into `relationship` collection.

In [14]:
rnid_start = daqa_dbmanip.get_collectionNewIdMax(
    daqa_scrap.relationship_coll, "relationshipid"
)
daqa_scrap.update_firm_relationships(fr_id_start=rnid_start + 1)

firm_relationship_update:  34%|██████████████████▏                                  | 310/904 [00:00<00:00, 633.02it/s]


#### Add (archobj) `article`' Relationships

In [15]:
updated_colls = [
    daqa_scrap.firm_coll,
    daqa_scrap.person_coll,
    daqa_scrap.interview_coll,
    daqa_scrap.stru_coll,
]
rnid_start = daqa_dbmanip.get_collectionNewIdMax(
    daqa_scrap.relationship_coll, "relationshipid"
)
# daqa_scrap.update_articlesRelations(updated_colls, ar_id_start=rnid_start + 1)

#### Add Media Relationships

In [16]:
rnid_start = daqa_dbmanip.get_collectionNewIdMax(
    daqa_scrap.relationship_coll, "relationshipid"
)
daqa_scrap.update_media_relationships(mr_id_start=rnid_start+1)

100%|█████████████████████████████████████████████████████████████████████████████| 6639/6639 [00:57<00:00, 115.97it/s]


#### Add Interview-Person Relationships

In [17]:
pirnid_start = daqa_dbmanip.get_collectionNewIdMax(
    daqa_scrap.relationship_coll, "relationshipid"
)

In [18]:
daqa_scrap.update_person_interview_relationships(pir_id_start=pirnid_start + 1)

person_interview_relationship_update: 97it [00:00, 607.88it/s]                                                         


#### Add Interview/Project-Place Relationships

In [19]:
related_places_pipelines = {
    "structure": {
        "_id": 0,
        "label": 1,
        "type": "place",
        "geo_coord": {"latitude": "$latitude", "longitude": "$longitude",},
        "address": {
            "country": "$coord_loc_details.address.country",
            "state": "$coord_loc_details.address.state",
            "city": "$coord_loc_details.address.city",
            "suburb": "$coord_loc_details.address.suburb",
            "postcode": "$coord_loc_details.address.postcode",
            "ori_address": "$location",
        },
        "relationships": {
            "subject": "$id",
            "subject_dbid": "$_id",
            "subjectlabel": "$label",
            "subjecttype": "$type",
            "objectlabel": "$label",
            "objecttype": "place",
            "relationship": "LocatedIn",
        },
    },
    "interview": {
        "_id": 0,
        "label": "$location",
        "type": "place",
        "address": {"ori_address": "$location",},
        "relationships": {
            "subject": "$id",
            "subject_dbid": "$_id",
            "subjectlabel": "$label",
            "subjecttype": "$type",
            "objectlabel": "$location",
            "objecttype": "place",
            "relationship": "DoneIn",
        },
    },
}

new_pId = 1
for curr_coll, curr_projection in related_places_pipelines.items():

    new_rId = (
        daqa_dbmanip.get_collectionNewIdMax(
            daqa_scrap.daqa_db["relationship"], "relationshipid"
        )
        + 1
    )
    print(new_rId)

    with tqdm(
        total=daqa_scrap.daqa_db[curr_coll].count_documents(
            {"location": {"$exists": True, "$nin": ["", None]}}
        ),
        desc=f"related_place_in_{curr_coll}_extraction",
    ) as pbar:

        for r in daqa_scrap.daqa_db[curr_coll].aggregate(
            [
                {"$match": {"location": {"$exists": True, "$nin": ["", None]}}},
                {"$project": {"relationships": 0}},
                {"$project": curr_projection},
            ]
        ):
            r.update({"id": f"{new_pId}_acde"})
            relations = r.get("relationships")
            place_inserted_id = daqa_scrap.daqa_db["place"].insert_one(r).inserted_id
            relations.update(
                {
                    "object": f"{new_pId}_acde",
                    "object_dbid": place_inserted_id,
                    "relationshipid": f"{new_rId}_acde",
                }
            )
            daqa_scrap.daqa_db["relationship"].insert_one(relations).inserted_id
            relations = {"relationships": [relations]}
            daqa_scrap.daqa_db["place"].update_one(
                {"_id": place_inserted_id}, {"$set": relations}
            )
            new_pId += 1
            new_rId += 1
            pbar.update(1)

6713


related_place_in_structure_extraction: 100%|██████████████████████████████████████| 1843/1843 [00:03<00:00, 595.14it/s]


8556


related_place_in_interview_extraction: 100%|██████████████████████████████████████████| 77/77 [00:00<00:00, 548.07it/s]


### Add Unpublished (Archival) Objects in Relationships into Collections

In [20]:
for obj_type in daqa_scrap.all_obj_coll.find({"category": "archobj"}).distinct("type"):
    daqa_scrap.update_missObjInRelationship(obj_type)

All article objects in relationships are stored.
All award objects in relationships are stored.
All education objects in relationships are stored.
All event objects in relationships are stored.


update_missing_firm: 100%|██████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 136.73it/s]


All government objects in relationships are stored.


update_missing_interview: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.13it/s]

All organisation objects in relationships are stored.



update_missing_person: 100%|████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 150.41it/s]


All place objects in relationships are stored.
All publication objects in relationships are stored.


update_missing_structure: 100%|███████████████████████████████████████████████████████| 28/28 [00:00<00:00, 178.71it/s]

All topic objects in relationships are stored.


### Add ori_dbid in relationship collection

In [21]:
# from general import MongoDBManipulation as gen_manip

# gen_manip.mdb_remove_fields(
#     daqa_scrap.daqa_db, "relationship", {}, ["subject_dbid", "object_dbid"]
# )

In [22]:
ori_dbid_mapping = defaultdict(dict)
for coll_name in set(daqa_scrap.daqa_db.list_collection_names()) - set(
    ["all_objects", "relationship"]
):
    for record in daqa_scrap.daqa_db[coll_name].find(
        {}, {"ori_dbid": "$_id", "id": 1, "type": 1, "_id": 0}
    ):
        ori_dbid_mapping[record.get("type")][record.get("id")] = record.get("ori_dbid")

In [23]:
removed_relat = []
with tqdm(
    total=daqa_scrap.relationship_coll.count_documents({}),
    desc="add_entity_ori_dbid_relationship",
) as pbar:
    for relation_r in daqa_scrap.relationship_coll.find({}):
        new_relation_r = copy.copy(relation_r)
        for entity_type in ["object", "subject"]:
            e_t_type = relation_r[f"{entity_type}type"]
            e_t_id = relation_r.get(entity_type)
            e_t_ori_dbid = ori_dbid_mapping[e_t_type].get(e_t_id)
            if e_t_ori_dbid is None:
                removed_relat.append(relation_r)
                break
            new_relation_r[f"{entity_type}_dbid"] = e_t_ori_dbid
            daqa_scrap.relationship_coll.update_one(
                {"_id": relation_r["_id"]}, {"$set": new_relation_r}
            )
        pbar.update(1)

add_entity_ori_dbid_relationship: 100%|█████████████████████████████████████████| 17549/17549 [00:21<00:00, 821.96it/s]


## 4. Clean Date Format Fields

In [3]:
daqa_scrap.daqa_udpate_date_format()

The following date fields are updated:
article: published, pubts, created
firm: start, end, pubts, created
structure: completion, pubts, created
award: pubts, created
publication: pubts, created
place: pubts, created
event: pubts, created
organisation: pubts, created
government: pubts, created
media: created
interview: pubts, created
topic: pubts, created
education: pubts, created
person: pubts, created


## 5. Update New Fields

In [4]:
updated_bio = pd.read_excel(
    os.path.join(data_folder, "exported_csv", "DAQA_BIOGRAPHY5_dvdp.xlsx"), nrows=848
).rename(
    {
        "first_school": "school1",
        "second_school": "school2",
        "third_school": "school3",
        "Unnamed: 21": "school4",
        "Unnamed: 22": "location4",
        "Unnamed: 23": "sch_type4",
        "Unnamed: 24": "graduation_year4",
        "Unnamed: 25": "award_qualification4",
        "reg_date2": "regis_date2",
    },
    axis=1,
)
updated_bio = updated_bio[
    ~updated_bio.applymap(
        lambda x: True if x in ("DELETE", "delete", "Delete") else False
    ).any(axis=1)
]
for idx, i in updated_bio[
    updated_bio.id.apply(lambda x: False if isinstance(x, int) else True)
][["id", "label"]].iterrows():
    query_rsp = daqa_scrap.daqa_db['person'].find_one({'label': i["label"]}, {"_id": 0, "id": 1})
    updated_bio.loc[idx, 'id'] = query_rsp.get("id")
id_columns = [
    "id",
]
updated_columns = [
    "gender",
    "school1",
    "location1",
    "sch_type1",
    "graduation_year1",
    "award_qualification1",
    "school2",
    "location2",
    "sch_type2",
    "graduation_year2",
    "award_qualification2",
    "school3",
    "location3",
    "sch_type3",
    "graduation_year3",
    "award_qualification3",
    "school4",
    "location4",
    "sch_type4",
    "graduation_year4",
    "award_qualification4",
    "career_start/firm_start",
    "regis_date1",
    "regis_location1",
    "regis_date2",
    "regis_location2",
    "career_ongoing",
    "retirement/last_project_year",
    "birth_year",
    "death_year",
    "career_focus1",
    "career_focus2",
    "influence_on_QLD_architecture",
]
# data cleaning
updated_bio = updated_bio[updated_bio.gender.isin(["male", "female"])][
    id_columns + updated_columns
]
updated_bio = updated_bio.fillna("nan").applymap(
    lambda x: "nan" if x in ["na", "?", " ", "NA?", "N/a", "N/A"] else x
)
updated_bio.loc[updated_bio.sch_type2 == 1961] = [
    584,
    "male",
    "Budapest",
    "Hungary",
    "UNI",
    "incomplete",
    'nan',
    "UNSW",
    "NSW",
    "UNI",
    1961,
    "BArch",
    "Liverpool",
    "UK",
    "UNI",
    1968,
    "MArch",
    "UQ",
    "QLD",
    "UNI",
    1978,
    "PhD",
    1961,
    1975,
    "QLD",
    "nan",
    "nan",
    "nan",
    "nan",
    1927,
    "nan",
    "nan",
    "nan",
    "nan",
]
for col in updated_bio:
    if col.startswith(("school", "award_qualification", "regis_location", "career_focus")):
        updated_bio[col] = updated_bio[col].apply(lambda x: x.strip())
    elif col.startswith("sch_type"):
        updated_bio[col] = updated_bio[col].apply(lambda x: 'INDEPENDENT' if x == 'Indep' else x.strip().upper())
    elif col.startswith("graduation_year"):
        updated_bio[col] = updated_bio[col].apply(lambda x: 'incomplete' if x in ('incomp', 'incomp') else x)
    if ('_year' in col) or ("_date" in col):
        updated_bio[col] = updated_bio[col].apply(lambda x: int(x) if isinstance(x, str) and x.isdigit() else x)
        
updated_bio = updated_bio.applymap(
    lambda x: None if x in ('nan', 'NAN') else x
).fillna('Unknown')

In [5]:
with tqdm(total=updated_bio.shape[0], desc="update_biogarphy") as pbar:
    for pid, r in updated_bio.set_index(["id"]).iterrows():
        final_r = defaultdict(list)
        final_r["id"] = pid
        final_r["gender"] = r["gender"]
        final_r["birth"] = {
            "coverage": {
                "date": {
                    "year": str(int(r["birth_year"]))
                    if r["birth_year"] != "Unknown"
                    else None
                }
            }
        }
        final_r["death"] = {
            "coverage": {
                "date": {
                    "year": str(int(r["death_year"]))
                    if r["death_year"] != "Unknown"
                    else None
                }
            }
        }
        final_r["influence_on_QLD_architecture"] = (
            r["influence_on_QLD_architecture"]
            if r["influence_on_QLD_architecture"] != "Unknown"
            else None
        )
        final_r["career"] = {
            "is_ongoing": r["career_ongoing"]
            if r["career_ongoing"] != "Unknown"
            else None,
            "date_start": r["career_start/firm_start"]
            if r["career_start/firm_start"] != "Unknown"
            else None,
            "date_end": r["retirement/last_project_year"]
            if r["retirement/last_project_year"] != "Unknown"
            else None,
            "registrations": [],
            "career_focus": [],
            "career_periods": [
                {
                    "occupation": {"type": "architect", "title": "architect"},
                    "coverage_range": {
                        "date_range": {
                            "date_start": {
                                "year": r["career_start/firm_start"]
                                if r["career_start/firm_start"] != "Unknown"
                                else None
                            },
                            "date_end": {
                                "year": r["retirement/last_project_year"]
                                if r["retirement/last_project_year"] != "Unknown"
                                else None
                            },
                        }
                    },
                }
            ],
        }
        for edu_idx in range(1, 5):
            edu_comp = r[
                [
                    f"school{edu_idx}",
                    f"location{edu_idx}",
                    f"sch_type{edu_idx}",
                    f"graduation_year{edu_idx}",
                    f"award_qualification{edu_idx}",
                ]
            ]
            if edu_comp.unique().tolist() == ["Unknown"]:
                pass
            else:
                edu_comp = edu_comp.to_dict()
                edu_comp = {
                    "organization": {
                        "name": edu_comp[f"school{edu_idx}"]
                        if edu_comp[f"school{edu_idx}"] != "Unknown"
                        else None,
                        "type": edu_comp[f"sch_type{edu_idx}"]
                        if edu_comp[f"sch_type{edu_idx}"] != "Unknown"
                        else None,
                        "qualification": edu_comp[f"award_qualification{edu_idx}"]
                        if edu_comp[f"award_qualification{edu_idx}"] != "Unknown"
                        else None,
                    },
                    "coverage_range": {
                        "place": edu_comp[f"location{edu_idx}"]
                        if edu_comp[f"location{edu_idx}"] != "Unknown"
                        else None,
                        "date_range": {
                            "date_start": None,
                            "date_end": {
                                "year": str(edu_comp[f"graduation_year{edu_idx}"])
                                if edu_comp[f"graduation_year{edu_idx}"] != "Unknown"
                                else None,
                            },
                        },
                    },
                }
                final_r["education_trainings"].append(edu_comp)
        for focus_idx in range(1, 3):
            focus_colname = f"career_focus{focus_idx}"
            if r[focus_colname] != "Unknown":
                final_r["career"][f"career_focus"].append(r[focus_colname])
        for regis_idx in range(1, 3):
            regis_comp = r[[f"regis_location{regis_idx}", f"regis_date{regis_idx}"]]
            if regis_comp.unique().tolist() == ["Unknown"]:
                pass
            else:
                regis_comp = regis_comp.to_dict()
                regis_comp = {
                    "coverage": {
                        "date": {
                            "year": str(regis_comp[f"regis_date{regis_idx}"])
                            if regis_comp[f"regis_date{regis_idx}"] != "Unknown"
                            else None
                        },
                        "place": regis_comp[f"regis_location{regis_idx}"]
                        if regis_comp[f"regis_location{regis_idx}"] != "Unknown"
                        else None,
                    }
                }
                final_r["registrations"].append(regis_comp)
        daqa_scrap.person_coll.update_one({"id": pid}, {"$set": final_r})
        pbar.update(1)

update_biogarphy: 100%|█████████████████████████████████████████████████████████████| 823/823 [00:04<00:00, 200.26it/s]


## Learning Notes:

[Python list of dictionaries search](https://stackoverflow.com/questions/8653516/python-list-of-dictionaries-search)

[jsonl2json](https://stackoverflow.com/questions/50475635/loading-jsonl-file-as-json-objects)

[json2jsonl](https://stackoverflow.com/questions/38915183/python-conversion-from-json-to-jsonl)

[Can‘t connect to HTTPS URL because the SSL module is not available](https://blog.csdn.net/Sky_Tree_Delivery/article/details/109078288)